In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/housing.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/housing.csv'

In [ ]:
print(data)

In [ ]:
print(data.dropna(inplace=True))
print(data)

In [ ]:
data.info

In [ ]:
X = data.drop(['median_house_value'],axis=1)
y = data['median_house_value']

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

train_data = X_train.join(y_train)
print(train_data)

In [ ]:
train_data.hist(figsize = (15,8))
plt.show()

In [ ]:
plt.figure(figsize=(14,9))
sns.heatmap(train_data.corr(),annot=True,cmap='YlGnBu')

In [ ]:
#since some features are right skewed we standardise them
train_data['total_rooms'] = np.log(train_data[ 'total_rooms']+1)
train_data['total_bedrooms'] = np.log(train_data[ 'total_bedrooms']+1)
train_data['population'] = np.log(train_data[ 'population']+1)
train_data['households'] = np.log(train_data[ 'households']+1)


In [ ]:
train_data.hist(figsize = (15,8))
plt.show()

In [ ]:
train_data

In [ ]:
print(train_data.ocean_proximity.value_counts())

In [ ]:
# Convert the ocean_proximity feature to a numeric format
ocean_numeric = pd.get_dummies(train_data.ocean_proximity).astype(int)
#print(ocean_numeric)

train_data = train_data.join(ocean_numeric).drop(['ocean_proximity'],axis=1)
print(train_data)

In [ ]:
plt.figure(figsize=(14,9))
sns.heatmap(train_data.corr(),annot=True,cmap='YlGnBu')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(x="latitude",y="longitude",data=train_data,hue='median_house_value',palette='coolwarm')
#the lower part which is red is the coastal area which shows that it has higher price than inland

In [ ]:
train_data['bedroom_ratio'] = train_data['total_bedrooms']/train_data['total_rooms']
train_data['household_bedrooms']=train_data['total_rooms']/train_data['households']

In [ ]:
plt.figure(figsize=(14,9))
sns.heatmap(train_data.corr(),annot=True,cmap='YlGnBu')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train,y_train = train_data.drop(['median_house_value'],axis=1), train_data['median_house_value']
X_train_s = scaler.fit_transform(X_train)


reg = LinearRegression()
reg.fit(X_train_s,y_train)

In [ ]:
test_data = X_test.join(y_test)

test_data['total_rooms'] = np.log(test_data[ 'total_rooms']+1)
test_data['total_bedrooms'] = np.log(test_data[ 'total_bedrooms']+1)
test_data['population'] = np.log(test_data[ 'population']+1)
test_data['households'] = np.log(test_data[ 'households']+1)

# Convert the ocean_proximity feature to a numeric format
ocean_numeric1 = pd.get_dummies(test_data.ocean_proximity).astype(int)
#print(ocean_numeric)

test_data = test_data.join(ocean_numeric1).drop(['ocean_proximity'],axis=1)


test_data['bedroom_ratio'] = test_data['total_bedrooms']/test_data['total_rooms']
test_data['household_bedrooms']=test_data['total_rooms']/test_data['households']
print(test_data)


In [ ]:
#train_data.drop(['ISLAND'],axis=1,inplace=True)
train_data


In [ ]:
X_test,y_test = test_data.drop(['median_house_value'],axis=1), test_data['median_house_value']

In [ ]:
X_test_s = scaler.fit_transform(X_test)

In [ ]:
reg.score(X_test_s,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()

forest.fit(X_train,y_train)

In [ ]:
forest.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [30,50,100],
    'max_features': [8,12,20]
}
grid_search = GridSearchCV(forest,param_grid,cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_train_s,y_train)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
best_forest = grid_search.best_estimator_

In [ ]:
best_forest.score(X_test_s,y_test)